# Non-negative Matrix Factorization (CMF)
## Init

In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
from functools import partial

import plotly.express as px


# import implicit # Matrix Factorization

from sklearn.decomposition import NMF
from sklearn.model_selection import KFold
from cmfrec import CMF_implicit

In [2]:
import sys
sys.path.append("../../")
from src.data_preprocessing import TrainTestGenerator
from src.evaluator import Evaluator

In [3]:
show_ploty = False

In [4]:
data_dir = "../../data/"
data_generator = TrainTestGenerator(data_dir)

## Model

In [5]:
# Model wrapper

class CMF_recommender:
    def __init__(self, k=32):
        self.model = CMF_implicit(
            # verbose=False,
            # method="als",
            nonneg=True,
            k=k,
            random_state=1,
            niter=100,
            # lambda_=1e+1
        )

    def fit(self, data: pd.DataFrame):
        data = data.copy()
        data = data.rename(columns={
            "userID": "UserId",
            "artistID": "ItemId",
            "weight": "Rating"
        })
        self.model.fit(data)

    def recommend(self, user_id, n):
        recommendations = self.model.topN(user_id, n=n)
        return recommendations

## Evaluation

In [6]:
# Evaluator (forward chaining)

evaluator = Evaluator(CMF_recommender, data_generator)
evaluator.evaluate()

evaluator.save_results("../../results/cmf_nonneg_ranks.csv", "../../results/cmf_nonneg_times.csv")

In [7]:
# Hit Rate

evaluator.get_hit_rates()

,cases,5,10,25,50,500
2008,4556,0.006365,0.013828,0.025461,0.039728,0.265803
2009,4687,0.006187,0.011948,0.024536,0.040324,0.225944
2010,6133,0.009946,0.018262,0.044676,0.068482,0.295940


In [8]:
# Mean Reciprocal Rank

evaluator.get_mrr()

,cases,mrr
2008,2608,0.013333
2009,3086,0.009288
2010,4306,0.015252


In [9]:
# Times

evaluator.get_times()

,count,mean,std,min,25%,50%,75%,max
task,,,,,,,,
model_fit,3.0,1.212870,0.276162,0.912737,1.091184,1.269631,1.362936,1.456240
model_init,3.0,0.000022,0.000010,0.000013,0.000017,0.000021,0.000027,0.000034
recommend_user,2336.0,0.000814,0.001484,0.000376,0.000476,0.000649,0.000755,0.039649


In [10]:
evaluator.get_fit_per_year_times()

,tag,time
task,,
model_fit,model_fit_2008,0.912737
model_fit,model_fit_2009,1.269631
model_fit,model_fit_2010,1.456240
